In [1]:
#pip install clean-text

In [2]:
import numpy as np
import pandas as pd
import re
import dask
import dask.dataframe as dd
from dask.diagnostics import ProgressBar
from dask.distributed import Client, LocalCluster
from pyvi import ViPosTagger, ViTokenizer
import string
from wordcloud import WordCloud
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

### Load File

In [3]:
df1= pd.read_csv("Products_ThoiTrangNam_raw.csv", encoding="utf-8")

In [4]:
df1.head()

,product_id,product_name,category,sub_category,link,image,price,rating,description
0,190,"Áo ba lỗ thun gân ,form body tôn dáng",Thời Trang Nam,Áo Ba Lỗ,https://shopee.vn/%C3%81o-ba-l%E1%BB%97-thun-g...,https://cf.shopee.vn/file/2c1ca03f5dc42f316fdf...,86250.0,4.9,Danh Mục\nShopee\nThời Trang Nam\nÁo Ba Lỗ\nCh...
1,191,"Áo Ba Lỗ Nam Trắng Chất Cotton Siêu Mát, Siêu Đẹp",Thời Trang Nam,Áo Ba Lỗ,https://shopee.vn/%C3%81o-Ba-L%E1%BB%97-Nam-Tr...,https://cf.shopee.vn/file/c7ea4c6574dc79be6b26...,26800.0,4.9,Danh Mục\nShopee\nThời Trang Nam\nÁo Ba Lỗ\nXu...
2,192,"Áo Ba Lỗ Nam Tyasuo chất vải co dãn mát, không...",Thời Trang Nam,Áo Ba Lỗ,https://shopee.vn/%C3%81o-Ba-L%E1%BB%97-Nam-Ty...,https://cf.shopee.vn/file/6f93bcda10efe374f8cc...,39500.0,4.8,Danh Mục\nShopee\nThời Trang Nam\nÁo Ba Lỗ\nTh...
3,193,ÁO BA LỖ HÀNG VIỆT NAM 100% COTTON,Thời Trang Nam,Áo Ba Lỗ,https://shopee.vn/%C3%81O-BA-L%E1%BB%96-H%C3%8...,https://cf.shopee.vn/file/1d7ed5e34bff8bc8b49a...,16500.0,4.8,Danh Mục\nShopee\nThời Trang Nam\nÁo Ba Lỗ\nCh...
4,194,Áo Thun Nam Thể Thao Ba Lỗ Mẫu Mới Siêu Đẹp (B...,Thời Trang Nam,Áo Ba Lỗ,https://shopee.vn/%C3%81o-Thun-Nam-Th%E1%BB%83...,NaN,45000.0,4.8,Danh Mục\nShopee\nThời Trang Nam\nÁo Ba Lỗ\nCh...


In [5]:
df1.shape

(49663, 9)

In [6]:
df1.iloc[3]['description']

'Danh Mục\nShopee\nThời Trang Nam\nÁo Ba Lỗ\nChất liệu\nCotton\nKho hàng\n7981\nGửi từ\nHà Nội\n\n--------------------Áo Ba Lỗ Nam CAO CẤP 100% Cotton Mặc nhà-----------------------\nHàng chất liệu 100% cotton, mặt vải mịn, thoáng khí, co giãn tốt, có khả năng thấm hút nước cao, thoát nước nhanh khô, đem lại cảm giác thoải mái, tự tin cho người sử dụng, quy trình xử lý vải an toàn, không độc hại, không gây kích ứng da.\nÁo có thể mặc lót trong sơ mi hay mặc đi thể thao,trong nhà đều được. Kiểu giáng đơn giản phù hợp với đại đa số.\n\nÁo có size M, L, XL (2L), XXL (3L)\nSize M : ngực 85-93, cao 160-165, 50-60kg\nSize L: ngực 93-101, cao 165-170, 60-65kg\nSize 2L: ngực 101-109, cao 170-175, 66-75kg\n\n#aothun #aobalo #chatcotton'

### Check NaN

In [7]:
df1['description'].isnull().sum()

963

In [8]:
df1 = df1[~df1['description'].isnull()]

In [9]:
df1.shape

(48700, 9)

In [10]:
df1['sub_category'].value_counts()

Đồ Bộ                        5095
Trang Phục Truyền Thống      5070
Khác                         5005
Vớ/Tất                       4937
Đồ Hóa Trang                 4893
Cà vạt & Nơ cổ               4626
Kính Mắt Nam                 4162
Áo Khoác                     1616
Quần Short                   1500
Quần Jeans                   1500
Áo Ba Lỗ                     1500
Đồ Ngủ                       1500
Áo Hoodie, Áo Len & Áo Nỉ    1500
Đồ Lót                       1500
Quần Dài/Quần Âu             1498
Áo                           1434
Áo Vest và Blazer            1364
Name: sub_category, dtype: int64

In [11]:
df0 = df1[df1['sub_category']== 'Đồ Bộ'].sample(frac=0.2)
df2 = df1[df1['sub_category']== 'Trang Phục Truyền Thống'].sample(frac=0.2)
df3 = df1[df1['sub_category']== 'Khác'].sample(frac=0.2)
df4 = df1[df1['sub_category']== 'Vớ/Tất'].sample(frac=0.2)
df5 = df1[df1['sub_category']== 'Đồ Hóa Trang'].sample(frac=0.2)
df6 = df1[df1['sub_category']== 'Cà vạt & Nơ cổ'].sample(frac=0.25)
df7 = df1[df1['sub_category']== 'Kính Mắt Nam'].sample(frac=0.25)
df8 = df1[df1['sub_category']== 'Áo Khoác'].sample(frac=0.7)
df9 = df1[df1['sub_category']== 'Quần Short'].sample(frac=0.7)
df10 = df1[df1['sub_category']== 'Quần Jeans'].sample(frac=0.7)
df11 = df1[df1['sub_category']== 'Áo Ba Lỗ'].sample(frac=0.7)
df12 = df1[df1['sub_category']== 'Đồ Ngủ'].sample(frac=0.7)
df13 = df1[df1['sub_category']== 'Áo Hoodie, Áo Len & Áo Nỉ'].sample(frac=0.7)
df14 = df1[df1['sub_category']== 'Đồ Lót'].sample(frac=0.7)
df15 = df1[df1['sub_category']== 'Quần Dài/Quần Âu'].sample(frac=0.7)
df16 = df1[df1['sub_category']== 'Áo'].sample(frac=0.7)
df17 = df1[df1['sub_category']== 'Áo Vest và Blazer'].sample(frac=0.8)

In [12]:
frames = [df0,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,df13,df14,df15,df16,df17]
df_new = pd.concat(frames)

In [13]:
df_new['sub_category'].value_counts()

Cà vạt & Nơ cổ               1156
Áo Khoác                     1131
Áo Vest và Blazer            1091
Quần Jeans                   1050
Đồ Lót                       1050
Áo Hoodie, Áo Len & Áo Nỉ    1050
Đồ Ngủ                       1050
Áo Ba Lỗ                     1050
Quần Short                   1050
Quần Dài/Quần Âu             1049
Kính Mắt Nam                 1040
Đồ Bộ                        1019
Trang Phục Truyền Thống      1014
Áo                           1004
Khác                         1001
Vớ/Tất                        987
Đồ Hóa Trang                  979
Name: sub_category, dtype: int64

In [14]:
df_new.reset_index(drop=True, inplace=True)

In [15]:
df_new.shape

(17771, 9)

In [16]:
df_new.iloc[7]['description']

'Danh Mục\nShopee\nThời Trang Nam\nĐồ Bộ\nMẫu\nIn\nKho hàng\n244342\nGửi từ\nQuận 12, TP. Hồ Chí Minh\n\nLƯU Ý:CÁC BẠN ĐẶT HÀNG THÀNH CÔNG, SHOP SẼ GỌI ĐIỆN LẠI TƯ VẤN SEZI ÁO CỤ THỂ KHÁCH NHÉ! \n\nCÁC BẠN VÀO PHẦN MUA HÀNG ĐẶT SỐ LƯỢNG 01 LÀ ĐƯỢC 2 ÁO.\n\n Sản phẩm có 6 size S, M , L ,XL ,XXL ,XXXL ,Freesize Nam Nữ.\n\nSize: S-M-L XL-XXL-XXL \n\nSize : S Dành cho bạn dưới 49 kg trở xuống cao dưới `1 m 60\n\nSize: m Dành cho bạn 57 kg trở xuống cao dưới1 m 70\n\nSize: L Dành cho bạn  68 kg trở xuống cao dưới 1M 75\n\nSize : XL Dành cho bạn 75 kg trở xuống\n\nSize : XXL Dành cho bạn  dưới 85 kg \n\nSize : XXXL Dành cho bạn dưới 95kg \n\n\nChất liệu : thun cotton không pha bất kỳ tạp chất mềm mịn co giãn tốt , tạo cho người mặc cảm giác cực kỳ thoải mái khi mặc . chất thun có tinh năng thấm hut môt hôi tốt ngay cả khi hoạt động ngoài trời , các bạn có thể phối hợp với , quần jean , chân váy , dày thể thao , theo phong cách cực teen\n\nLƯU Ý SHOP CHỈ BÁN HÀNG LOẠI 1 KHÔNG BÁN HÀNG LOẠI 2 

### PreProcess

In [17]:
from xulyvni import *

In [18]:
# Create a cluster
cluster = LocalCluster()
client = Client(cluster)
dask.config.set(scheduler='multiprocessing')

2023-03-29 14:44:39,201 - distributed.diskutils - INFO - Found stale lock file and directory '/var/folders/6v/nymwmjk912d8glyc17sfhddr0000gn/T/dask-worker-space/worker-xpth607h', purging
2023-03-29 14:44:39,201 - distributed.diskutils - INFO - Found stale lock file and directory '/var/folders/6v/nymwmjk912d8glyc17sfhddr0000gn/T/dask-worker-space/worker-9ms7l088', purging
2023-03-29 14:44:39,202 - distributed.diskutils - INFO - Found stale lock file and directory '/var/folders/6v/nymwmjk912d8glyc17sfhddr0000gn/T/dask-worker-space/worker-6zkkj07b', purging
2023-03-29 14:44:39,203 - distributed.diskutils - INFO - Found stale lock file and directory '/var/folders/6v/nymwmjk912d8glyc17sfhddr0000gn/T/dask-worker-space/worker-kdh194xa', purging


In [19]:
# Create a dask dataframe from the pandas dataframe
ddf = dd.from_pandas(df_new, npartitions=16)
ddf = ddf.map_partitions(xuly_vni)

In [20]:
# Convert the dask dataframe to pandas dataframe
print('Converting dask dataframe to pandas dataframe...')
with ProgressBar():
    df = ddf.compute(num_workers=7)

# Free up memory
del ddf

Converting dask dataframe to pandas dataframe...
[########################################] | 100% Completed | 221.11 s


In [21]:
#t1 = xuly_vni(df_t)

In [22]:
df['description'].isnull().sum()

0

In [23]:
df.iloc[500]['description']

'đồ nam phụ kiện thời đơn giản không thể thể mặc đồ nam đồ thể tập mát mẻ thoải mái giới thời phái set thể quần áo nam chiếm vị trí quan chàng thoải mái dạo phố hẹn hò bè đồ thể nam trở thành không thể mẹo giúp bảo quản quần áo nam đối sản phẩm quần áo giặt tiên màu quần áo giặt lộn mặt trái đảm bảo độ bền sản phẩm phù hợp giặt máy giặt không giặt đồ trắng đồ tối màu thông xuất xứ hàng hoá sở sản xuất đức chủ sở đức địa xóm xã trực nội huyện trực nam định thành vải linen thông kỹ thuật màu sắc đen trắng thông cảnh báo tất sản phẩm thời giặt sử dụng đảm bảo toàn hướng sử dụng hướng bảo quản không móc dị dạng phơi quần áo giặt lộn trái quần áo bền không giặt chất tẩy rửa sản xuất kết sản phẩm mô tả kiểu dáng hoàn toàn ảnh mẫu quần áo kiểm kĩ cẩn thận tư vấn nhiệt tình gói hàng quý hàng sẵn hàng đơn hoàn tiền sản phẩm không mô tả chấp đổi hàng không hỗ trợ đổi định màn hình kiện ánh màu sắc thực tế sản phẩm thể chênh lệch'

In [24]:
df.head()

,product_id,product_name,category,sub_category,link,image,price,rating,description
0,231475,Bộ Quần Áo Thể Thao Thời Trang Nam Q5SHOP Form...,Thời Trang Nam,Đồ Bộ,https://shopee.vn/B%E1%BB%99-Qu%E1%BA%A7n-%C3%...,https://cf.shopee.vn/file/e96fe533edf419d544c7...,79000.0,4.9,giới thiệu quần áo kết hình ảnh chụp không đền...
1,232886,SÉT ĐỒ THỂ THAO NAM🚗 MẪU MONTER ĐỘC ĐÁO HÀNG B...,Thời Trang Nam,Đồ Bộ,https://shopee.vn/S%C3%89T-%C4%90%E1%BB%92-TH%...,https://cf.shopee.vn/file/b76fe4944cb0b0e6bb96...,59000.0,4.7,quần áo thể dãn mùa hè chất lạnh mẫu bảng sét ...
2,231261,Đồ Bộ hè nam 🚚 FREESHIP 🚚 Quần áo thể thao nam...,Thời Trang Nam,Đồ Bộ,https://shopee.vn/%C4%90%E1%BB%93-B%E1%BB%99-h...,https://cf.shopee.vn/file/b00adb38c9ec0584a434...,49000.0,4.7,ớng cân cân chất liệu vải mè thái giãn thoải m...
3,234333,ĐỒ BỘ NAM MẪU MỚI PHỐI 2 MÀU CỰC XINH 2021,Thời Trang Nam,Đồ Bộ,https://shopee.vn/%C4%90%E1%BB%92-B%E1%BB%98-N...,https://cf.shopee.vn/file/5e67f8abf497d148ae82...,69000.0,4.7,thị trường sỉ chào quý thông sản phẩm lưu thiế...
4,233867,Bộ nam mùa hè chất mát cổ tròn cộc tay,Thời Trang Nam,Đồ Bộ,https://shopee.vn/B%E1%BB%99-nam-m%C3%B9a-h%C3...,https://cf.shopee.vn/file/513604fee1ae595669da...,150000.0,5.0,sản phẩm thể nam thể nam thiết kế form chuẩn n...


In [25]:
df.to_csv('project2_shopee_final.csv')

2023-03-29 18:23:52,674 - tornado.application - ERROR - Exception in callback <bound method SystemMonitor.update of <SystemMonitor: cpu: 8 memory: 55 MB fds: 27>>
Traceback (most recent call last):
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/tornado/ioloop.py", line 921, in _run
    val = self.callback()
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/distributed/system_monitor.py", line 156, in update
    net_ioc = psutil.net_io_counters()
  File "/Library/Frameworks/Python.framework/Versions/3.8/lib/python3.8/site-packages/psutil/__init__.py", line 2114, in net_io_counters
    rawdict = _psplatform.net_io_counters()
OSError: [Errno 12] Cannot allocate memory
